<a href="https://colab.research.google.com/github/mharrisonbaker/samplework/blob/master/patgraph2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00


In [2]:
!pip install -q dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 44.8 MB/s eta 0:00:00


In [3]:
#import common packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import random

#for graphs
import networkx as nx
import dgl

#for bigquery
import pandas_gbq
from google.oauth2 import service_account

#for BERT
import torch
from transformers import AutoTokenizer, AutoModel

#for word2vec
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec

#for learning
from sklearn.model_selection import train_test_split


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
%cd /content/drive/My Drive/oauth/
oauthpath="oauth1.json"

/content/drive/My Drive/oauth


In [6]:
credentials = service_account.Credentials.from_service_account_file(
    oauthpath,
)

In [7]:
project_id = 'patgraphcode'

dialect = {
    'project_id': project_id,
    'dialect': 'standard',
}

In [8]:
# Define the SQL query
# query = """
# SELECT
#   publication_number,
#   cpc_expanded.code AS cpc_code,
#   cpc_expanded.first as cpc_first,
# FROM
#   patents-public-data.google_patents_research.publications
# CROSS JOIN
#   UNNEST(cpc) AS cpc_expanded
# WHERE
#   publication_number LIKE 'US-%'
#   AND cpc_expanded.first = True
#   AND NOT publication_number LIKE 'US-D%'
#   AND (cpc_expanded.code LIKE 'G16B%' OR cpc_expanded.code LIKE 'G16H%')

# LIMIT
#   50
# """

# query = """
# SELECT DISTINCT
#   p.publication_number,
#   p.title,
#   p.abstract,
#   c.code AS cpc_code,
#   c.inventive AS cpc_inventive,
#   c.first AS cpc_first,
#   cb.publication_number AS cited_by_publication_number,
#   cb.application_number AS cited_by_application_number,
#   cb.npl_text AS cited_by_npl_text,
#   cb.type AS cited_by_type,
#   cb.category AS cited_by_category,
#   cb.filing_date AS cited_by_filing_date
# FROM
#   patents-public-data.google_patents_research.publications AS p,
#   UNNEST(p.cited_by) AS cb
# LEFT JOIN UNNEST(p.cpc) AS c
# ON TRUE
# WHERE
#   p.publication_number LIKE 'US-%'
#   AND NOT p.publication_number LIKE 'US-D%'
#   AND (c.code LIKE 'G10L%')
#   AND c.inventive = TRUE

# LIMIT 100000
# """

# AND (c.code LIKE 'G16B%' OR c.code LIKE 'G16H%')

query = """
SELECT DISTINCT
  p1.publication_number AS publication_number,
  p1.title AS title,
  p1.abstract AS abstract,
  c.code AS cpc_code,
  c.inventive AS cpc_inventive,
  c.first AS cpc_first,
  p2.publication_number AS cited_by_publication_number,
  p2.title AS cited_by_title,
  p2.abstract AS cited_by_abstract,
  c2.code AS cited_by_cpc_code,
  c2.inventive AS cited_by_cpc_inventive,
  c2.first AS cited_by_cpc_first
FROM
  patents-public-data.google_patents_research.publications AS p1,
  UNNEST(p1.cited_by) AS cb
LEFT JOIN UNNEST(p1.cpc) AS c
ON TRUE
LEFT JOIN patents-public-data.google_patents_research.publications AS p2
ON p2.publication_number = cb.publication_number
LEFT JOIN UNNEST(p2.cpc) AS c2
ON TRUE
WHERE
  p1.publication_number LIKE 'US-%'
  AND NOT p1.publication_number LIKE 'US-D%'
  AND (c.code LIKE 'G10L%' OR c2.code LIKE 'G10L%')
  AND (c.inventive = TRUE OR c2.inventive = TRUE)
LIMIT 5000
"""

In [9]:
#read SQL result into pandas df
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)

In [10]:
#normalize Publication numbers
df['publication_number'] = df['publication_number'].str.replace(r'^US-|-[A-Z][0-9]$', '', regex=True)
df['publication_number'] = df['publication_number'].str.replace(r'-[A-Z]$', '', regex=True)
df['cited_by_publication_number'] = df['cited_by_publication_number'].str.replace(r'^US-|-[A-Z][0-9]$', '', regex=True)
df['cited_by_publication_number'] = df['cited_by_publication_number'].str.replace(r'-[A-Z]$', '', regex=True)

In [11]:
df.head(10)

,publication_number,title,abstract,cpc_code,cpc_inventive,cpc_first,cited_by_publication_number,cited_by_title,cited_by_abstract,cited_by_cpc_code,cited_by_cpc_inventive,cited_by_cpc_first
0,2002161646,Advertising campaign and business listing mana...,The present invention discloses a method and s...,H04W4/20,True,False,9263039,Systems and methods for responding to natural ...,Systems and methods are provided for receiving...,G10L15/19,True,False
1,2016260049,"Shopping facility assistance systems, devices,...",A plurality of motorized terrestrial transport...,G06F3/017,True,False,10570000,Shopping facility assistance object detection ...,Some embodiments provide apparatuses and metho...,G10L15/22,True,False
2,2016260145,Multi-shopper cooperative shopping experience ...,Methods and apparatuses are provided for use i...,G10L15/22,True,False,10570000,Shopping facility assistance object detection ...,Some embodiments provide apparatuses and metho...,G06F18/214,True,False
3,8175730,Device and method for analyzing an information...,"In order to analyze an information signal, a s...",G10L25/48,True,True,10318503,Insight and algorithmic clustering for automat...,"A decision support system and method, which re...",G06F16/248,True,False
4,7949529,Mobile systems and methods of supporting natur...,A mobile system is provided that includes spee...,G10L15/1815,True,True,8688099,Open development system for access service pro...,A method comprising configuring and installing...,G06Q20/32,True,False
5,7869996,Recognition of speech in editable audio streams,A speech processing system divides a spoken au...,G10L15/22,True,True,10665241,Rapid frontend resolution of transcription-rel...,Being able to rapidly and accurately transcrib...,G10L15/22,True,False
6,2019156822,Multiple turn conversational task assistance,A technique for multiple turn conversational t...,G10L25/78,True,False,10665241,Rapid frontend resolution of transcription-rel...,Being able to rapidly and accurately transcrib...,G10L2015/0631,False,False
7,2018342256,System and Method for Voice-to-Voice Conversion,A method of building a speech conversion syste...,G10L15/22,True,False,11600284,Voice morphing apparatus having adjustable par...,A voice morphing apparatus having adjustable p...,H04K1/04,True,False
8,2018342256,System and Method for Voice-to-Voice Conversion,A method of building a speech conversion syste...,G10L13/033,True,False,11600284,Voice morphing apparatus having adjustable par...,A voice morphing apparatus having adjustable p...,G06N20/00,True,False
9,7475015,Semantic language modeling and confidence meas...,A system and method for speech recognition inc...,G10L15/1815,True,True,10810274,Optimizing dialogue policy decisions for digit...,Systems and processes for optimizing dialogue ...,H04N21/42203,True,False


In [ ]:
# df2=df

In [12]:
# #filter to G10L?
# #write function for HEC encoding function
# def encode_hec(code):
#     parts = code.split(' ')
#     main_category = parts[0]
#     sub_category = parts[1] if len(parts) > 1 else ''
#     hierarchy_code = f'{main_category} - 100'

#     if sub_category:
#         sub_parts = sub_category.split('/')
#         for i, sub_part in enumerate(sub_parts):
#             hierarchy_code += f'\n {sub_part} - {bin(i + 1)[2:].zfill(7)}'

#     return hierarchy_code

# # Apply HEC encoding to the classification_code column
# df2['hec_encoding'] = df2['cpc_code'].apply(encode_hec)

NameError: ignored

In [ ]:
#df2.head(10)

In [13]:
columns_to_check = ['title', 'abstract', 'cpc_code']
missing_values = df[columns_to_check].isnull().sum()
print(missing_values)

title       0
abstract    0
cpc_code    0
dtype: int64


In [25]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [14]:
# Check if the data folder exists, create it if it doesn't
if not os.path.exists('data'):
    os.makedirs('data')

In [15]:
# Define the path to the pickle file
pickle_path = 'data/citation_sql_data.pickle'


In [16]:
import torch
from transformers import AutoTokenizer, AutoModel

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the SciBERT model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Move the model to the device
model = model.to(device)

# Batch size for generating embeddings
batch_size = 32

# Assuming you have a DataFrame named df with an 'abstract' column
embeddings = []

# Iterate over each batch of the DataFrame
for batch_start in range(0, len(df), batch_size):
    batch_df = df.iloc[batch_start:batch_start+batch_size]
    batch_texts = batch_df['abstract'].tolist()

    # Tokenize the batch texts
    tokens = tokenizer.batch_encode_plus(batch_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    # Generate the embeddings
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = outputs.pooler_output.cpu().numpy().tolist()

    embeddings.extend(batch_embeddings)

# Add the embeddings as a new column in the DataFrame
df['abstract_embedding'] = embeddings




cited_by_embeddings = []

# Iterate over each batch of the DataFrame
for batch_start in range(0, len(df), batch_size):
    batch_df = df.iloc[batch_start:batch_start+batch_size]
    batch_texts = batch_df['cited_by_abstract'].tolist()

    # Tokenize the batch texts
    tokens = tokenizer.batch_encode_plus(batch_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    # Generate the embeddings
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = outputs.pooler_output.cpu().numpy().tolist()

    cited_by_embeddings.extend(batch_embeddings)

# Add the embeddings as a new column in the DataFrame
df['cited_by_abstract_embedding'] = cited_by_embeddings



Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [82]:
device

device(type='cuda')

In [17]:
# Initialize the title embeddings columns
df['title_embedding'] = None
df['cited_by_title_embedding'] = None

# Prepare batches for the 'title' column
for batch_start in range(0, len(df), batch_size):
    batch_df = df.iloc[batch_start:batch_start+batch_size]
    batch_texts = batch_df['title'].tolist()

    # Tokenize the batch texts
    tokens = tokenizer.batch_encode_plus(batch_texts, truncation=True, padding=True, max_length=64, return_tensors='pt')
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    # Generate the embeddings
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = outputs.pooler_output.cpu().numpy()

    # Add the embeddings to the DataFrame
    for i, embedding in enumerate(batch_embeddings):
        df.at[batch_start+i, 'title_embedding'] = embedding

# Prepare batches for the 'cited_by_title' column
for batch_start in range(0, len(df), batch_size):
    batch_df = df.iloc[batch_start:batch_start+batch_size]
    batch_texts = batch_df['cited_by_title'].tolist()

    # Tokenize the batch texts
    tokens = tokenizer.batch_encode_plus(batch_texts, truncation=True, padding=True, max_length=64, return_tensors='pt')
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    # Generate the embeddings
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_embeddings = outputs.pooler_output.cpu().numpy()

    # Add the embeddings to the DataFrame
    for i, embedding in enumerate(batch_embeddings):
        df.at[batch_start+i, 'cited_by_title_embedding'] = embedding

In [18]:
df.head(3)

,publication_number,title,abstract,cpc_code,cpc_inventive,cpc_first,cited_by_publication_number,cited_by_title,cited_by_abstract,cited_by_cpc_code,cited_by_cpc_inventive,cited_by_cpc_first,abstract_embedding,cited_by_abstract_embedding,title_embedding,cited_by_title_embedding
0,2002161646,Advertising campaign and business listing mana...,The present invention discloses a method and s...,H04W4/20,True,False,9263039,Systems and methods for responding to natural ...,Systems and methods are provided for receiving...,G10L15/19,True,False,"[0.005589909385889769, -0.09646502137184143, -...","[0.10368426144123077, -0.11612911522388458, -0...","[-0.021623997, -0.067425795, -0.3458059, 0.602...","[0.07478379, 0.04013107, 0.4316531, 0.99628156..."
1,2016260049,"Shopping facility assistance systems, devices,...",A plurality of motorized terrestrial transport...,G06F3/017,True,False,10570000,Shopping facility assistance object detection ...,Some embodiments provide apparatuses and metho...,G10L15/22,True,False,"[-0.17310114204883575, -0.3296165466308594, -0...","[0.06949799507856369, -0.07625707238912582, 0....","[-0.0135331685, 0.1939859, 0.66813517, 0.93190...","[-0.14899431, -0.01520236, -0.6003777, 0.66281..."
2,2016260145,Multi-shopper cooperative shopping experience ...,Methods and apparatuses are provided for use i...,G10L15/22,True,False,10570000,Shopping facility assistance object detection ...,Some embodiments provide apparatuses and metho...,G06F18/214,True,False,"[0.053884685039520264, -0.20862428843975067, -...","[0.06949799507856369, -0.07625707238912582, 0....","[-0.39618537, -0.4554069, -0.92999244, 0.62208...","[-0.14899431, -0.01520236, -0.6003777, 0.66281..."


In [19]:
# Assuming you have a DataFrame named df with 'cpc_code' and 'cited_by_cpc_code' columns

# Define the hierarchical levels of CPC codes
cpc_levels = ['section', 'class', 'subclass']

# Function to split the CPC code into hierarchical levels
def split_cpc_code(cpc_code):
    if cpc_code is None:
        return []  # Return an empty list or any other appropriate value for missing values
    return cpc_code.split('/')

# Create a list to hold the hierarchical embeddings
cpc_embeddings = []
cited_by_cpc_embeddings = []

# Train embeddings for each level of the hierarchy
for level in cpc_levels:
    # Collect codes at the current level
    cpc_codes = df['cpc_code'].apply(split_cpc_code).apply(lambda x: x[:cpc_levels.index(level) + 1]).values.tolist()
    cited_by_cpc_codes = df['cited_by_cpc_code'].apply(split_cpc_code).apply(lambda x: x[:cpc_levels.index(level) + 1]).values.tolist()

    # Train Word2Vec embeddings for the current level
    embedding_dim = 100  # Adjust the dimensionality as needed
    model = Word2Vec(sentences=cpc_codes + cited_by_cpc_codes, vector_size=embedding_dim, min_count=1)

    # Extract embeddings for the codes at the current level
    cpc_level_embeddings = np.array([model.wv[code[-1]] for code in cpc_codes])
    cited_by_cpc_level_embeddings = np.array([model.wv[code[-1]] if code else np.zeros(embedding_dim) for code in cited_by_cpc_codes])


    # Append the embeddings to the hierarchical embeddings list
    cpc_embeddings.append(cpc_level_embeddings)
    cited_by_cpc_embeddings.append(cited_by_cpc_level_embeddings)

# Combine the hierarchical embeddings
cpc_combined_embeddings = np.concatenate(cpc_embeddings, axis=1)
cited_by_cpc_combined_embeddings = np.concatenate(cited_by_cpc_embeddings, axis=1)

# Add the embeddings to the DataFrame
for i, level in enumerate(cpc_levels):
    df[f'cpc_embedding_{level}'] = cpc_combined_embeddings[:, i]
    df[f'cited_by_cpc_embedding_{level}'] = cited_by_cpc_combined_embeddings[:, i]


In [20]:
df.head(3)

,publication_number,title,abstract,cpc_code,cpc_inventive,cpc_first,cited_by_publication_number,cited_by_title,cited_by_abstract,cited_by_cpc_code,...,abstract_embedding,cited_by_abstract_embedding,title_embedding,cited_by_title_embedding,cpc_embedding_section,cited_by_cpc_embedding_section,cpc_embedding_class,cited_by_cpc_embedding_class,cpc_embedding_subclass,cited_by_cpc_embedding_subclass
0,2002161646,Advertising campaign and business listing mana...,The present invention discloses a method and s...,H04W4/20,True,False,9263039,Systems and methods for responding to natural ...,Systems and methods are provided for receiving...,G10L15/19,...,"[0.005589909385889769, -0.09646502137184143, -...","[0.10368426144123077, -0.11612911522388458, -0...","[-0.021623997, -0.067425795, -0.3458059, 0.602...","[0.07478379, 0.04013107, 0.4316531, 0.99628156...",0.005627,-0.000536,0.005497,0.000236,0.001829,0.005103
1,2016260049,"Shopping facility assistance systems, devices,...",A plurality of motorized terrestrial transport...,G06F3/017,True,False,10570000,Shopping facility assistance object detection ...,Some embodiments provide apparatuses and metho...,G10L15/22,...,"[-0.17310114204883575, -0.3296165466308594, -0...","[0.06949799507856369, -0.07625707238912582, 0....","[-0.0135331685, 0.1939859, 0.66813517, 0.93190...","[-0.14899431, -0.01520236, -0.6003777, 0.66281...",0.008132,-0.000536,-0.004457,0.000236,-0.001068,0.005103
2,2016260145,Multi-shopper cooperative shopping experience ...,Methods and apparatuses are provided for use i...,G10L15/22,True,False,10570000,Shopping facility assistance object detection ...,Some embodiments provide apparatuses and metho...,G06F18/214,...,"[0.053884685039520264, -0.20862428843975067, -...","[0.06949799507856369, -0.07625707238912582, 0....","[-0.39618537, -0.4554069, -0.92999244, 0.62208...","[-0.14899431, -0.01520236, -0.6003777, 0.66281...",-0.000536,-0.006155,0.000236,0.004668,0.005103,-0.009071


In [21]:
# Save the data to the pickle file
with open(pickle_path, 'wb') as f:
    pickle.dump(df, f)

In [22]:
with open(pickle_path, 'rb') as f:
    data = pickle.load(f)

In [23]:
G = nx.Graph()
# df = pd.DataFrame(data)

In [24]:
df.columns

Index(['publication_number', 'title', 'abstract', 'cpc_code', 'cpc_inventive',
       'cpc_first', 'cited_by_publication_number', 'cited_by_title',
       'cited_by_abstract', 'cited_by_cpc_code', 'cited_by_cpc_inventive',
       'cited_by_cpc_first', 'abstract_embedding',
       'cited_by_abstract_embedding', 'title_embedding',
       'cited_by_title_embedding', 'cpc_embedding_section',
       'cited_by_cpc_embedding_section', 'cpc_embedding_class',
       'cited_by_cpc_embedding_class', 'cpc_embedding_subclass',
       'cited_by_cpc_embedding_subclass'],
      dtype='object')

In [25]:
for index, row in df.iterrows():
    # Add/Update the 'publication_number' node with attributes
    G.add_node(row['publication_number'],
               title_embedding=row['title_embedding'],
               abstract_embedding=row['abstract_embedding'],
               cpc_embedding_section=row['cpc_embedding_section'],
               cpc_embedding_class=row['cpc_embedding_class'],
               cpc_embedding_subclass=row['cpc_embedding_subclass'])

    # Add 'cited_by_publication_number' node, with attributes
    G.add_node(row['cited_by_publication_number'],
               title_embedding=row['cited_by_title_embedding'],
               abstract_embedding=row['cited_by_abstract_embedding'],
               cpc_embedding_section=row['cited_by_cpc_embedding_section'],
               cpc_embedding_class=row['cited_by_cpc_embedding_class'],
               cpc_embedding_subclass=row['cited_by_cpc_embedding_subclass'])

    # Add an edge between 'publication_number' and 'cited_by_publication_number'
    G.add_edge(row['publication_number'], row['cited_by_publication_number'])


In [26]:
# Save the graph to a pickle file
with open('graph.pickle', 'wb') as f:
    pickle.dump(G, f)

In [27]:
#load graph from pickle file
with open('graph.pickle', 'rb') as f:
    G = pickle.load(f)


In [49]:
#check a node to see if graph is good
node_id='2016260145'
G.has_node(node_id)
attrs = G.nodes[node_id]
print(f"Node {node_id} Attributes: {attrs}")

Node 2016260145 Attributes: {'title_embedding': array([-0.39618585, -0.45540705, -0.9299923 ,  0.6220836 ,  0.25978002,
        0.99968153,  0.20933196, -0.9974765 ,  0.53605914,  0.18715121,
        0.17151648,  0.13014339,  0.9894276 ,  0.06817875, -0.19469558,
        0.20862944, -0.17747048, -0.25118774, -0.20682316, -0.602822  ,
       -0.31031564,  0.405848  ,  0.87640905,  0.29180655, -0.9674287 ,
        0.9838087 , -0.25083825,  0.31712335, -0.18120505,  0.5302158 ,
        0.15454932,  0.3369794 ,  0.1941378 ,  0.06280427,  0.8532592 ,
        0.16565974,  0.9995285 ,  0.20269758,  0.17377299,  0.8689562 ,
       -0.09373106,  0.2999521 , -0.1301769 , -0.08186144,  0.13658533,
       -0.2858503 , -0.3040646 ,  0.27025738,  0.04794158, -0.96505266,
        0.02201525, -0.48025256, -0.20818433, -0.8423846 ,  0.61962014,
       -0.485169  , -0.36312923,  0.57681274, -0.93235147,  0.93130434,
        0.01297793, -0.62802064,  0.0728514 ,  0.96604943,  0.27863955,
       -0.100855

In [83]:
# Convert NetworkX graph to DGL graph
dgl_graph = dgl.from_networkx(G, node_attrs=['cpc_embedding_section', 'cpc_embedding_class', 'abstract_embedding', 'cpc_embedding_subclass', 'title_embedding'], edge_attrs= None, device='cuda')

DGLError: ignored

In [54]:
# # Get a list of attribute names from the NetworkX graph
# attribute_names = []
# for node in G.nodes:
#     attribute_names.extend(list(G.nodes[node].keys()))
# attribute_names = list(set(attribute_names))

# # Transfer node attributes from NetworkX graph to DGL graph
# for attr_name in attribute_names:
#     attr_values = [G.nodes[node][attr_name] for node in G.nodes]
#     dgl_graph.ndata[attr_name] = torch.tensor(attr_values)


<ipython-input-54-8ccdf544c5fb>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  dgl_graph.ndata[attr_name] = torch.tensor(attr_values)


In [55]:
# print(attribute_names)

['cpc_embedding_section', 'cpc_embedding_class', 'abstract_embedding', 'cpc_embedding_subclass', 'title_embedding']


In [78]:
num_nodes = dgl_graph.num_nodes()
print("Number of nodes:", num_nodes)

Number of nodes: 6459


In [79]:
num_nodes = len(G.nodes)
print("Number of nodes:", num_nodes)

Number of nodes: 6459


In [80]:
num_edges_networkx = G.number_of_edges()
print("Number of edges in NetworkX graph:", num_edges_networkx)

Number of edges in NetworkX graph: 4932


In [81]:
num_edges_dgl = dgl_graph.number_of_edges()
print("Number of edges in DGL graph:", num_edges_dgl)

Number of edges in DGL graph: 9864


In [61]:
# Access the node attributes in the DGL graph
node_attributes = dgl_graph.ndata
print(node_attributes)


{'cpc_embedding_section': tensor([-0.0053, -0.0005,  0.0081,  ..., -0.0005, -0.0043, -0.0005]), 'cpc_embedding_class': tensor([-0.0021,  0.0002, -0.0045,  ...,  0.0002, -0.0093,  0.0002]), 'abstract_embedding': tensor([[ 0.0056, -0.0965, -0.5627,  ..., -0.4930,  0.8543, -0.9500],
        [ 0.1037, -0.1161, -0.9476,  ..., -0.5556,  0.9956, -0.9666],
        [-0.1731, -0.3296, -0.1335,  ..., -0.3297,  0.8403, -0.2837],
        ...,
        [ 0.0257, -0.3962, -0.9080,  ..., -0.4149,  0.9888,  0.5013],
        [ 0.2807,  0.2665,  0.5572,  ..., -0.4100,  0.8826, -0.9831],
        [-0.1060, -0.2139,  0.5024,  ..., -0.4412,  0.9821,  0.7310]]), 'cpc_embedding_subclass': tensor([ 0.0041,  0.0051, -0.0011,  ...,  0.0051, -0.0019,  0.0051]), 'title_embedding': tensor([[-0.0216, -0.0674, -0.3458,  ..., -0.3294, -0.9059, -0.0875],
        [ 0.0748,  0.0401,  0.4317,  ..., -0.0502,  0.9913,  0.0979],
        [-0.0135,  0.1940,  0.6681,  ..., -0.3304,  0.9898, -0.8934],
        ...,
        [ 0.2723

In [62]:
# Step 1: Randomly select a subset of nodes for labeling
num_labeled_nodes = 100  # Number of nodes to be labeled
labeled_nodes = np.random.choice(dgl_graph.num_nodes(), size=num_labeled_nodes, replace=False)

# Step 2: Define the number of classes
num_classes = 10  # Replace with the actual number of classes

# Step 3: Generate random labels for the labeled nodes
labels = np.random.randint(low=0, high=num_classes, size=num_labeled_nodes)

# Step 4: Split labeled nodes into training and validation sets
train_nodes, val_nodes, train_labels, val_labels = train_test_split(labeled_nodes, labels, test_size=0.2, random_state=42)

# Step 5: Assign node labels in the DGL graph
dgl_graph.ndata['labels'] = torch.zeros(dgl_graph.num_nodes(), dtype=torch.long)  # Initialize labels with zeros
dgl_graph.ndata['labels'][train_nodes] = torch.tensor(train_labels, dtype=torch.long)
dgl_graph.ndata['labels'][val_nodes] = torch.tensor(val_labels, dtype=torch.long)


In [64]:
# Accessing node properties
print(dgl_graph.ndata['title_embedding'][1])


tensor([ 7.4784e-02,  4.0131e-02,  4.3165e-01,  9.9628e-01,  2.9517e-01,
         9.9816e-01,  1.6048e-01, -9.9654e-01,  2.7007e-01, -3.4464e-01,
        -1.0792e-02,  4.6950e-01, -2.3674e-01,  7.1440e-01, -4.0233e-01,
         2.6059e-01, -9.5225e-02, -6.0671e-01,  7.5124e-02, -9.7876e-01,
         1.0006e-02,  4.8181e-02, -9.1596e-01,  9.4549e-02,  7.4210e-01,
        -4.0864e-01, -9.9208e-01,  1.5792e-01,  3.7125e-02,  1.2753e-01,
         5.2912e-01, -9.8088e-01,  3.2101e-01, -5.8896e-01, -5.8496e-01,
        -4.3342e-02,  9.9854e-01, -9.3180e-02,  2.2086e-01,  4.3807e-01,
        -4.4054e-01,  2.7001e-01,  8.5079e-01, -4.5571e-01,  2.6710e-01,
        -1.0918e-01, -3.5952e-01,  1.2994e-01, -9.8074e-01,  9.7509e-01,
         2.4020e-01, -3.3806e-01, -4.6676e-02,  7.4940e-01,  9.9945e-01,
        -3.1942e-02, -4.5082e-01,  2.0103e-01, -9.9130e-01,  3.8543e-01,
        -2.0011e-02, -9.9519e-01,  4.3981e-01, -3.2887e-01,  1.1757e-01,
        -1.8621e-01, -4.0621e-01, -2.9184e-01,  9.0

In [65]:
print(df['title_embedding'].dtype)

object


In [66]:
# Get the embeddings as arrays from the DataFrame
title_embeddings = df['title_embedding'].values
abstract_embeddings = df['abstract_embedding'].values
cpc_section_embeddings = df['cpc_embedding_section'].values
cpc_class_embeddings = df['cpc_embedding_class'].values
cpc_subclass_embeddings = df['cpc_embedding_subclass'].values

# Convert the array embeddings to numerical tensors
title_embeddings = torch.from_numpy(np.stack(title_embeddings))
abstract_embeddings = torch.from_numpy(np.stack(abstract_embeddings))
cpc_section_embeddings = torch.from_numpy(np.stack(cpc_section_embeddings))
cpc_class_embeddings = torch.from_numpy(np.stack(cpc_class_embeddings))
cpc_subclass_embeddings = torch.from_numpy(np.stack(cpc_subclass_embeddings))

# Verify the data type and shape of the tensors
print(title_embeddings.dtype)
print(title_embeddings.shape)
# Repeat the same for other embeddings

# Assign node features in the DGL graph
dgl_graph.ndata['title_embedding'] = title_embeddings
dgl_graph.ndata['abstract_embedding'] = abstract_embeddings
dgl_graph.ndata['cpc_embedding_section'] = cpc_section_embeddings
dgl_graph.ndata['cpc_embedding_class'] = cpc_class_embeddings
dgl_graph.ndata['cpc_embedding_subclass'] = cpc_subclass_embeddings




torch.float32
torch.Size([5000, 768])


DGLError: ignored

In [67]:
dgl_graph.num_nodes()

6459

In [ ]:
# Save the graph to a GraphML file
#nx.write_graphml(G, "test.graphml")

In [ ]:
all_have_title = all('title_embedding' in data for node, data in G.nodes(data=True))
print(all_have_title)

True


In [ ]:
count_without_title = sum(1 for node, data in G.nodes(data=True) if 'title_embedding' not in data)
print(count_without_title)

0


In [ ]:
# Convert to a DGL graph
# D = dgl.from_networkx(G, node_attrs=['title_embedding', 'cpc', 'abstract_embedding'])
D = dgl.from_networkx(G, node_attrs=['title_embedding', 'abstract_embedding'])

In [ ]:
num_edges = G.degree['6329925']
num_edges

99

In [ ]:
# num_edges_total = G.edges()
# num_edges_total
# spits out long code

In [ ]:
#take a look at the graph
n=5
degree_centrality = nx.degree_centrality(G)

# Sort the nodes by degree centrality in descending order
sorted_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)

# Print the top n nodes with highest degree centrality
for node in sorted_nodes[:n]:
    centrality = degree_centrality[node]
    print(f"Node {node}: Degree Centrality = {centrality}")


Node 2012016678: Degree Centrality = 0.006694450432324975
Node 11405466: Degree Centrality = 0.004089722401858638
Node 11152002: Degree Centrality = 0.0040777466407990225
Node 11500672: Degree Centrality = 0.004011879954971139
Node 11423886: Degree Centrality = 0.003963976910732677


In [ ]:
# Create an empty subgraph
subgraph = nx.Graph()

# Create a queue for BFS traversal
queue = deque()

# Add the starting node to the subgraph and enqueue it
subgraph.add_node(start_node)
queue.append((start_node, 0))

# Perform BFS up to three levels
while queue:
    node, level = queue.popleft()

    # Stop traversing further if reached the desired level (3 in this case)
    if level == 3:
        continue

    # Enqueue neighboring nodes at the next level
    for neighbor in G.neighbors(node):
        if neighbor not in subgraph:
            subgraph.add_node(neighbor)
            subgraph.add_edge(node, neighbor)
            queue.append((neighbor, level + 1))

# Visualize the subgraph
plt.figure(figsize=(8, 8))
nx.draw_networkx(subgraph)
plt.title("Connected Subgraph up to Three Levels")
plt.axis("off")
plt.show()

NameError: ignored

In [ ]:
#for undirected graphs
# Get the connected components
connected_components = list(nx.connected_components(G))

# Sort the connected components by size in descending order
connected_components.sort(key=len, reverse=True)

# Get the largest subgraph
largest_subgraph = connected_components[0]

# Calculate statistics
num_nodes = len(largest_subgraph)
largest_subgraph_view = G.subgraph(largest_subgraph)
num_edges = largest_subgraph_view.number_of_edges()
average_degree = sum(dict(largest_subgraph_view.degree()).values()) / num_nodes

# Print statistics
print("Statistics of the Largest Subgraph:")
print(f"Number of Nodes: {num_nodes}")
print(f"Number of Edges: {num_edges}")

print(f"Average Degree: {average_degree}")

Statistics of the Largest Subgraph:
Number of Nodes: 46887
Number of Edges: 82540
Average Degree: 3.520805340499499


In [ ]:
# # load word2vec model
# w2v_model = model = api.load('word2vec-google-news-300')
# # w2v_model = model = api.load('biowordvec-pubmed-and-wikipedia-50')

# # Iterate through each row in the DataFrame
# for index, row in df.iterrows():
#     abstract_text = row['abstract']
#     words = abstract_text.split()

#     # Retrieve the word2vec embeddings for each word in the abstract
#     embeddings = []
#     for word in words:
#         if word in model.key_to_index:
#             embeddings.append(model.get_vector(word))

#     # Average the embeddings to get a single embedding for the abstract
#     if embeddings:
#         abstract_embedding = np.mean(embeddings, axis=0)
#     else:
#         abstract_embedding = None

#     # Add the abstract_embedding as a new column in the DataFrame
#     df.at[index, 'abstract_embedding'] = abstract_embedding

In [ ]:
# #for directed graphs
# # perform the weakly connected component analysis
# weak_ccs = nx.weakly_connected_components(G)

# # get the largest connected component subgraphs by size
# largest_weak_ccs = sorted(weak_ccs, key=len, reverse=True)[:20]

# # print the size and nodes of each largest subgraph
# for i, cc in enumerate(largest_weak_ccs):
#     print(f"Largest subgraph #{i+1} has {len(cc)} nodes: {cc}")

In [ ]:
# nx.draw(G, with_labels=True)
# plt.show()

In [ ]:
# # Find the connected components in the graph
# connected_components = nx.connected_components(G)

# # Iterate over each connected component
# nodes_with_longest_chain = []
# for component in connected_components:
#     subgraph = G.subgraph(component)
#     diameter = nx.diameter(subgraph)

#     for node in subgraph.nodes():
#         shortest_path_length = nx.single_source_shortest_path_length(subgraph, node)
#         longest_path_length = max(shortest_path_length.values())
#         if longest_path_length == diameter:
#             nodes_with_longest_chain.append(node)

# print("Node(s) with the longest chain of edges:", nodes_with_longest_chain)
